In [37]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers, callbacks
from PIL import Image
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




In [31]:
# Paths
base_dir = r"datasets/tomato"
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

# Image size and parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = len(os.listdir(train_dir))  # auto-detect number of classes

# ======= 1. Distribusi Data per Kelas =======
print("\n📊 Distribusi data latih:")
for cls in os.listdir(train_dir):
    cls_path = os.path.join(train_dir, cls)
    print(f"{cls}: {len(os.listdir(cls_path))} gambar")

# ======= 1. Distribusi Data per Kelas =======
print("\n📊 Distribusi data validasi:")
for cls in os.listdir(val_dir):
    cls_path = os.path.join(val_dir, cls)
    print(f"{cls}: {len(os.listdir(cls_path))} gambar")


📊 Distribusi data latih:
Tomato___Bacterial_spot: 1000 gambar
Tomato___Early_blight: 1000 gambar
Tomato___healthy: 1000 gambar
Tomato___Late_blight: 1000 gambar
Tomato___Leaf_Mold: 1000 gambar
Tomato___Septoria_leaf_spot: 1000 gambar
Tomato___Spider_mites_Two_spotted_spider_mite: 1000 gambar
Tomato___Target_Spot: 1000 gambar
Tomato___Tomato_mosaic_virus: 1000 gambar
Tomato___Tomato_Yellow_Leaf_Curl_Virus: 1000 gambar

📊 Distribusi data validasi:
Tomato___Bacterial_spot: 100 gambar
Tomato___Early_blight: 100 gambar
Tomato___healthy: 100 gambar
Tomato___Late_blight: 100 gambar
Tomato___Leaf_Mold: 100 gambar
Tomato___Septoria_leaf_spot: 100 gambar
Tomato___Spider_mites_Two_spotted_spider_mite: 100 gambar
Tomato___Target_Spot: 100 gambar
Tomato___Tomato_mosaic_virus: 100 gambar
Tomato___Tomato_Yellow_Leaf_Curl_Virus: 100 gambar


In [33]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)
print("Class Indices:", train_gen.class_indices)
print("Class Indices:", val_gen.class_indices)


Found 10000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Class Indices: {'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___Late_blight': 2, 'Tomato___Leaf_Mold': 3, 'Tomato___Septoria_leaf_spot': 4, 'Tomato___Spider_mites_Two_spotted_spider_mite': 5, 'Tomato___Target_Spot': 6, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato___Tomato_mosaic_virus': 8, 'Tomato___healthy': 9}
Class Indices: {'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___Late_blight': 2, 'Tomato___Leaf_Mold': 3, 'Tomato___Septoria_leaf_spot': 4, 'Tomato___Spider_mites_Two_spotted_spider_mite': 5, 'Tomato___Target_Spot': 6, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato___Tomato_mosaic_virus': 8, 'Tomato___healthy': 9}


In [34]:
# Bangun model
def build_model():
    base_model = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SIZE + (3,),
        pooling='avg'
    )
    
    # Freeze base model untuk fine-tuning bertahap
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [39]:
# Callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,  # Berhenti jika tidak ada peningkatan dalam 5 epoch
    verbose=1,
    mode='max',
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_efficientnet_tomato.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)


In [41]:
# Training model
print("🚀 Memulai training model...")
model = build_model()

# Fase 1: Training hanya lapisan atas
history = model.fit(
    train_gen,
    epochs=25,
    validation_data=val_gen,
    callbacks=[early_stopping, checkpoint]
)

# Fase 2: Fine-tuning (unfreeze sebagian base model)
print("\n🔧 Memulai fine-tuning...")
model = tf.keras.models.load_model('best_efficientnet_tomato.h5')
model.trainable = True

# Unfreeze lapisan atas EfficientNet
for layer in model.layers[0].layers[-20:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Learning rate lebih rendah
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callback untuk fine-tuning
ft_early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    mode='max',
    restore_best_weights=True
)

ft_checkpoint = ModelCheckpoint(
    'finetuned_efficientnet_tomato.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# Training fine-tuning
history_ft = model.fit(
    train_gen,
    epochs=10,  # Maksimum 10 epoch untuk fine-tuning
    validation_data=val_gen,
    callbacks=[ft_early_stopping, ft_checkpoint]
)

# Evaluasi model akhir
print("\n📊 Evaluasi model akhir:")
model = tf.keras.models.load_model('finetuned_efficientnet_tomato.h5')
val_loss, val_acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {val_acc*100:.2f}%")


🚀 Memulai training model...
Epoch 1/25
313/313 [==============================] - ETA: 0s - loss: 2.3162 - accuracy: 0.0950
Epoch 1: val_accuracy improved from -inf to 0.10000, saving model to best_efficientnet_tomato.h5


c:\Users\anira\.conda\envs\tldpy3.10\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


313/313 [==============================] - 356s 1s/step - loss: 2.3162 - accuracy: 0.0950 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/25
313/313 [==============================] - ETA: 0s - loss: 2.3031 - accuracy: 0.0943
Epoch 2: val_accuracy did not improve from 0.10000
313/313 [==============================] - 252s 806ms/step - loss: 2.3031 - accuracy: 0.0943 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/25
313/313 [==============================] - ETA: 0s - loss: 2.3028 - accuracy: 0.0939
Epoch 3: val_accuracy did not improve from 0.10000
313/313 [==============================] - 252s 806ms/step - loss: 2.3028 - accuracy: 0.0939 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 4/25
313/313 [==============================] - ETA: 0s - loss: 2.3028 - accuracy: 0.0966
Epoch 4: val_accuracy did not improve from 0.10000
313/313 [==============================] - 250s 799ms/step - loss: 2.3028 - accuracy: 0.0966 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 5/25
313/313 [==

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000002934DC0B400>>
Traceback (most recent call last):
  File "c:\Users\anira\.conda\envs\tldpy3.10\lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [15]:
# ======= 3. Prediksi Gambar Input dan Visualisasi =======
def predict_and_plot(image_path):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    pred_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Visualisasi
    plt.imshow(load_img(image_path))
    plt.axis('off')
    plt.title(f"Prediction: {pred_class} ({confidence:.2f}%)")
    plt.show()